In [1]:
# Data を読み込み確認する
import pandas as pd
uselog = pd.read_csv('sample_code/chapter_4/use_log.csv')
uselog.isnull().sum() # 欠損値の確認

log_id         0
customer_id    0
usedate        0
dtype: int64

In [2]:
customer = pd.read_csv('sample_code/chapter_4/customer_join.csv')
customer.isnull().sum() # 欠損値の確認

customer_id             0
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
dtype: int64

end_date 以外は欠損値が 0 であることを確認。

顧客 Data を 利用履歴に基づいた Group 化を行う。
※あらかじめ決められた正解がない為、教師なし学習の Clustering を用いる。
